使用SimFin API获取美股价格信息 (pandas, requests, cache, dict; momentum)

- SimFin API documentation: https://simfin.com/api/v1/documentation/
- Officia Tutorial: https://medium.com/@SimFin_official/simfin-api-tutorial-6626c6c1dbeb
- 【蟹老板Python 101】：http://www.bossqiao.com/menu/index.php?p=coding
- 代码：

In [7]:
import pandas as pd
import requests
import requests_cache

global api_key,sim_id_dict,stock_price_dict
api_key = "rLvevB5Qhi5Ruu8Yab6zNzwdZ2Nu98cA"
sim_id_dict={}
stock_price_dict={}

#86400 seconds = 24 hours
requests_cache.install_cache('simfin_cache', backend='sqlite', expire_after=86400)

- build the sim_id dictionary

In [8]:
#https://simfin.com/api/v1/info/find-id/ticker/amd?api-key=rLvevB5Qhi5Ruu8Yab6zNzwdZ2Nu98cA

def sim_id_map(tickers):
    global sim_id_dict, api_key
    
    for ticker in tickers:
        request_url = f'https://simfin.com/api/v1/info/find-id/ticker/{ticker}?api-key={api_key}'
        content = requests.get(request_url)
        data = content.json()
        #print(data)
        if "error" in data or len(data) < 1:
            print("price data empty, 检查ticker是否拼错")
        else:
            sim_id_dict[data[0]['ticker']] = data[0]['simId']


tickers = ["AMD","NVDA","INTC","MU","VZ","DIS","BAC","WBA","DAL"]
sim_id_map(tickers)
print(sim_id_dict)

{'AMD': 191397, 'NVDA': 172199, 'INTC': 85652, 'MU': 60906, 'VZ': 101219, 'DIS': 261356, 'BAC': 71941, 'WBA': 87466, 'DAL': 231789}


- 通过sim_id获取股价信息

In [9]:
#https://simfin.com/api/v1/companies/id/191397/shares/prices?api-key=rLvevB5Qhi5Ruu8Yab6zNzwdZ2Nu98cA
#get stock price history
def get_price_info(tickers):
    global sim_id_dict, api_key, stock_price_dict

    for ticker in tickers:
        companyId = sim_id_dict[ticker]
        request_url = f'https://simfin.com/api/v1/companies/id/{companyId}/shares/prices?api-key={api_key}'
        price_content = requests.get(request_url)
        price_data = price_content.json()

        stock_price = pd.DataFrame(price_data['priceData'])
        stock_price["closeAdj"] = pd.to_numeric(stock_price["closeAdj"])
        stock_price = stock_price[["date", "closeAdj"]]
        stock_price_dict[ticker]=stock_price

get_price_info(tickers)
stock_price_dict['INTC'].head(10)

,date,closeAdj
0,2019-03-26,53.44
1,2019-03-25,52.78
2,2019-03-22,53.26
3,2019-03-21,54.64
4,2019-03-20,53.82
5,2019-03-19,54.17
6,2019-03-18,54.10
7,2019-03-15,54.33
8,2019-03-14,53.44
9,2019-03-13,54.37


In [10]:
#days=10代表过去10天的股价涨跌幅
def price_chg(ticker, days):
    global stock_price_dict
    price_now = stock_price_dict[ticker].iloc[0,1]
    try:
        price_past = stock_price_dict[ticker].iloc[days-1,1]
        price_chg = (price_now - price_past)/price_past
        price_chg = '%.1f%%' % (price_chg*100)
    except:
        price_chg = "N/A"
    return price_chg

ticker = "AMD"
days = 1000
print(price_chg(ticker, days))

844.5%


In [11]:
def show_momentum(tickers, periods):
    momentum = {}
    for ticker in tickers:
        list = []
        for period in periods:
            list.append(price_chg(ticker, period))
        momentum[ticker]=list

    df_m = pd.DataFrame(momentum, index=periods)
    return df_m

periods = [5, 10, 20, 30, 50, 100, 200, 300, 600, 1000, 3000]
df_m = show_momentum(tickers, periods)
df_m

,AMD,NVDA,INTC,MU,VZ,DIS,BAC,WBA,DAL
5,-0.0%,1.4%,-0.7%,0.5%,5.1%,0.1%,-5.0%,-1.4%,-1.6%
10,9.9%,4.9%,-1.7%,3.8%,5.2%,-3.5%,-7.1%,0.0%,-3.8%
20,9.4%,13.9%,0.4%,-2.4%,6.9%,-2.3%,-7.4%,-13.2%,-2.0%
30,12.6%,17.1%,6.9%,-0.2%,11.4%,0.9%,-4.7%,-12.9%,-0.9%
50,27.0%,17.7%,11.2%,16.3%,4.5%,-2.0%,3.7%,-11.8%,3.5%
100,49.4%,-12.7%,13.3%,12.0%,5.3%,-3.3%,2.7%,-20.5%,-9.7%
200,68.5%,-32.4%,-1.1%,-34.3%,27.4%,7.6%,-7.9%,-0.6%,-7.4%
300,109.2%,-19.3%,27.8%,-11.5%,22.8%,1.1%,-10.8%,-16.1%,-14.6%
600,252.4%,162.7%,67.0%,128.6%,43.6%,22.6%,72.3%,-19.1%,24.0%
1000,844.5%,733.5%,91.6%,51.1%,48.0%,10.7%,87.5%,-23.2%,27.2%


In [13]:
def get_price_date(ticker, date):
    try:
        df = stock_price_dict[ticker]
        return df[df['date']==date].iloc[0,1]
    except:
        print('no price info for ', ticker, date)
        
        
def price_chg_date(ticker, start_date, end_date):
    start_price = get_price_date(ticker, start_date)
    end_price = get_price_date(ticker, end_date)
    
    if((start_price is not None) and (end_price is not None)):
        price_chg = (end_price - start_price)/start_price
        price_chg = '%.1f%%' % (price_chg*100)
    else:
        price_chg = "N/A"
    return price_chg


def show_momentum_date(tickers, start_date, end_date):
    momentum = {}
    for ticker in tickers:
        momentum[ticker] = price_chg_date(ticker, start_date, end_date)
    df_m = pd.DataFrame(momentum, index=['%s%s%s' % (start_date, ' ~ ', end_date)])
    return df_m
        
#price_chg_var = price_chg('AMD', '2019-03-11', '2019-03-22')
#print(price_chg_var)

tickers = ["AMD","NVDA","INTC","MU","VZ","DIS","BAC","WBA","DAL"]
df1 = show_momentum_date(tickers, '2019-01-02', '2019-03-26')
df2 = show_momentum_date(tickers, '2018-01-02', '2019-03-26')
df3 = show_momentum_date(tickers, '2017-01-03', '2019-03-26')
pd.concat([df1, df2, df3])

,AMD,NVDA,INTC,MU,VZ,DIS,BAC,WBA,DAL
2019-01-02 ~ 2019-03-26,36.4%,30.0%,14.2%,23.1%,9.4%,1.1%,9.6%,-8.4%,-1.3%
2018-01-02 ~ 2019-03-26,134.0%,-10.9%,17.7%,-7.7%,19.9%,0.1%,-6.8%,-14.9%,-10.7%
2017-01-03 ~ 2019-03-26,124.8%,74.7%,54.9%,78.8%,23.3%,7.1%,25.6%,-21.6%,4.5%
